# Topic 8: Regularization

## Reading: Bishop 5.5

## 1 Motivating regularization

“YEAH, BUT YOUR SCIENTISTS WERE SO PREOCCUPIED WITH WHETHER OR NOT THEY COULD THAT THEY DIDN’T STOP TO THINK IF THEY SHOULD.” -- Dr. Ian Malcolm

<img src="images/goldblum.jpg">


We now have the power to create functions (namely neural networks) that have the power to approximate any other function, given a sufficient number of parameters.  However, as we learned when we were fitting polynomials to curves all the way back at the start of the class, unlimited model complexity is a path fraught with peril.  Recall that if we have a simple dataset like this one:  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (9,9)

np.random.seed(0)

# Create constantly-spaced x-values
x = np.linspace(0,1,11)

# Create a linear function of $x$ with slope 1, intercept 1, and normally distributed error with sd=1
y = x + np.random.randn(len(x))*0.1 + 1.0
y_test = x + np.random.randn(len(x))*0.1 + 1.0
plt.plot(x,y,'k.')
plt.plot(x,y_test,'r*')
plt.show()

We can fit arbitrarily complex models such that we hit the training data exactly

In [ ]:
plt.plot(x,y,'k.')
x_smooth = np.linspace(0,1,101)
train_errors = []
test_errors = []
for d in range(1,13,2):
    X = np.vander(x,d,increasing=True)
    w = np.linalg.solve(X.T @ X,X.T@y)
    y_pred = X@w
    plt.plot(x_smooth,np.vander(x_smooth,d,increasing=True)@w)
    training_error = 1./len(y)*np.sum((y_pred-y)**2)
    test_error = 1./len(y_test)*np.sum((y_pred-y_test)**2)
    train_errors.append(training_error)
    test_errors.append(test_error)
    
plt.show()

If we then plot the resulting training set and test errors, we see this typical pattern:

In [ ]:
plt.plot(train_errors,label='Training error')
plt.plot(test_errors,label='Test error')
plt.legend()
plt.show()    

Of course the same thing can happen in neural networks.  In the case of linear regression, we made the simple choice to just limit our model complexity.  This is certainly possible with neural nets.  However, it's a little bit more challenging to decide just exactly *where* overfitting is coming from, and to tailor the network in response.  Instead, we introduce a technique that we will refer to as *regularization*.  Regularization is broadly understood to be a technique that trades training set accuracy for test set accuracy, and there are a multitude of flavors, a few of which we explore here.  

## 2 L2 Regularization
The most common idea for regularizing has been around for a very long time, and it results from the observation that in most regression problems, large weights tend to correspond to overfitting the data.  As such, we can make an effort to reduce overfitting by explicitly penalizing large parameters in the cost function.  In particular, we can simply add the following term:
$$
\mathcal{L}' = \underbrace{\mathcal{L}}_{\text{Sum Square Error}} + \underbrace{\frac{\gamma}{2}\; \mathbf{w}^T \mathbf{w}}_{L_2-\text{norm of }\mathbf{w}}.
$$
It is straightforward to take the derivative of this augmented cost function with respect to the parameters, and setting them to zero yields an augmented set of normal equations:
$$
(X^T X + \gamma \mathcal{I}) \mathbf{w} = X^T y,
$$
where $\mathcal{I}$ is an appropriately sized identity matrix.  **What does the parameter $\gamma$ do**?

We can easily implement this for a high-dimensional problem, and explore what happens to our model fit as we adjust $\gamma$

In [ ]:
plt.plot(x,y,'k.')
x_smooth = np.linspace(0,1,101)
train_errors = []
test_errors = []
d = 13
gammas = np.logspace(-7,1,12)
for gamma in gammas:
    X = np.vander(x,d,increasing=True)
    identity = np.eye(X.shape[1])
    identity[0,0] = 0
    w = np.linalg.solve(X.T @ X + gamma*identity,X.T@y)
    y_pred = X@w
    plt.plot(x_smooth,np.vander(x_smooth,d,increasing=True)@w)
    training_error = 1./len(y)*np.sum((y_pred-y)**2)
    test_error = 1./len(y_test)*np.sum((y_pred-y_test)**2)
    train_errors.append(training_error)
    test_errors.append(test_error)
    
plt.show()

Once again, we can look at the test and training accuracies together.

In [ ]:
plt.semilogx(gammas,train_errors,label='Training error')
plt.semilogx(gammas,test_errors,label='Test error')
plt.legend()
plt.show()   

As similar picture emerges (although flipped around the x-axis, because a larger $\gamma$ corresponds to a simpler model.  

## L2 Regularization for neural networks

As it turns out, this is simple to apply to neural networks as well.  To illustrate its effect, let's synthesize a very noisy dataset to be used for a classification problem.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=300,noise=0.5)

X,X_test,y,y_test = train_test_split(X,y)
X0,y0 = X.copy(),y.copy()
X0_test,y0_test = X_test.copy(),y_test.copy()

plt.scatter(X[:,0],X[:,1],c=y)
plt.scatter(X_test[:,0],X_test[:,1],c=y_test,marker='x')

There is a clear pattern here, but it is quite noisy.  Let's see if we can fit a very flexible neural network to this dataset using pytorch.  We'll first go through our ritual of converting the data into the appropriate type and location, and then create a DataLoader for use with batch gradient descent.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

X = torch.from_numpy(X)
X_test = torch.from_numpy(X_test)
y = torch.from_numpy(y)
y_test = torch.from_numpy(y_test)


X = X.to(torch.float32)
X_test = X_test.to(torch.float32)
y = y.to(torch.long)
y_test = y_test.to(torch.long)

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

X = X.to(device)
X_test = X_test.to(device)
y = y.to(device)
y_test = y_test.to(device)

from torch.utils.data import TensorDataset

training_data = TensorDataset(X,y)
test_data = TensorDataset(X_test,y_test)

batch_size = 256
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

batch_size = 256
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=batch_size, 
                                           shuffle=False)

Now let's define our neural network.  It'll be a simple affair with a single hidden layer, but plenty of nodes to ensure a flexible function.  Something like the following:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        """
        This method is where you'll want to instantiate parameters.
        we do this by creating two linear transformation functions, l1 and l2, which 
        have encoded in it both the weight matrices W_1 and W_2, and the bias vectors
        """
        super(Net,self).__init__()
        self.l1 = nn.Linear(2,2048) # Transform from input to hidden layer
        self.l2 = nn.Linear(2048,2)
    
    def forward(self,x):
        """
        This method runs the feedforward neural network.  It takes a tensor of size m x 784,
        applies a linear transformation, applies a sigmoidal activation, applies the second linear transform 
        and outputs the logits.
        """
        a1 = self.l1(x)
        z1 = torch.relu(a1)
        
        a2 = self.l2(z1)
        return a2

In [ ]:
model = Net()
model.to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

optimizer = torch.optim.Adam(model.parameters())

epochs = 3000
# Loop over the data
for epoch in range(epochs):
    model.train()
    # Loop over each subset of data
    for d,t in train_loader:

        # Zero out the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Make a prediction based on the model
        outputs = model(d)
        
        # Compute the loss
        loss = criterion(outputs,t)      

        # Use backpropagation to compute the derivative of the loss with respect to the parameters
        loss.backward()
        
        # Use the derivative information to update the parameters
        optimizer.step()
        
    model.eval()
    # After each epoch, compute the test set accuracy
    total=0.
    correct=0.
    # Loop over all the test examples and accumulate the number of correct results in each batch
    for d,t in test_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total += float(t.size(0))
        correct += float((predicted==t).sum())
    total_train = 0
    correct_train = 0
    for d,t in train_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total_train += float(t.size(0))
        correct_train += float((predicted==t).sum())
        
    # Print the epoch, the training loss, and the test set accuracy.
    if epoch%10==0:
        print(epoch,loss.item(),100.*correct_train/total_train,100.*correct/total)

**The above case exhibits the classic symptoms of overfitting.  How do you know?**

This neural network only has two features, so we can easily visualize its predictions as a grid. Let's see what the decision boundary is.

In [ ]:
X0grid,X1grid = np.meshgrid(np.linspace(X[:,0].cpu().min(),X[:,0].cpu().max(),101),np.linspace(X[:,1].cpu().min(),X[:,1].cpu().max(),101))

X_grid = np.vstack((X0grid.ravel(),X1grid.ravel())).T
X_grid = torch.from_numpy(X_grid)
X_grid = X_grid.to(torch.float32)
X_grid = X_grid.cuda()

t = model(X_grid)
out = F.softmax(t,dim=1)
plt.contourf(X0grid,X1grid,out.cpu().detach().numpy()[:,1].reshape((101,101)),alpha=0.5)
plt.scatter(X0[:,0],X0[:,1],c=y0)
plt.scatter(X0_test[:,0],X0_test[:,1],c=y0_test,marker='x')

Obviously, the test set accuracy isn't so good because the model is learning a really crazy mapping from the features to the class probability!  We can allay this issue using L2 regularization.  How do we implement this?  As it turns out, it's not so bad.

In [ ]:
model = Net()
model.to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

optimizer = torch.optim.Adam(model.parameters())
gamma = 1.0  ### HERE'S THE REGULARIZATION PARAMETER!

epochs = 3000
# Loop over the data
for epoch in range(epochs):
    model.train()
    # Loop over each subset of data
    for d,t in train_loader:

        # Zero out the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Make a prediction based on the model
        outputs = model(d)
        
        # Compute the loss
        loss = criterion(outputs,t)
        
        ### HERE'S WHERE WE ADD REGULARIZATION!
        for p in model.parameters():
            # Loop over all the model parameters
            if p.dim()>1:
                # Loop over all the weight matrices (but not the biases)
                loss += gamma/(2*d.shape[0])*(p**2).sum()
        

        # Use backpropagation to compute the derivative of the loss with respect to the parameters
        loss.backward()
        
        # Use the derivative information to update the parameters
        optimizer.step()
        
    model.eval()
    # After each epoch, compute the test set accuracy
    total=0.
    correct=0.
    # Loop over all the test examples and accumulate the number of correct results in each batch
    for d,t in test_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total += float(t.size(0))
        correct += float((predicted==t).sum())
    total_train = 0
    correct_train = 0
    for d,t in train_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total_train += float(t.size(0))
        correct_train += float((predicted==t).sum())
        
    # Print the epoch, the training loss, and the test set accuracy.
    if epoch%10==0:
        print(epoch,loss.item(),100.*correct_train/total_train,100.*correct/total)

In [ ]:
X0grid,X1grid = np.meshgrid(np.linspace(X[:,0].cpu().min(),X[:,0].cpu().max(),101),np.linspace(X[:,1].cpu().min(),X[:,1].cpu().max(),101))

X_grid = np.vstack((X0grid.ravel(),X1grid.ravel())).T
X_grid = torch.from_numpy(X_grid)
X_grid = X_grid.to(torch.float32)
X_grid = X_grid.cuda()

t = model(X_grid)
out = F.softmax(t,dim=1)
plt.contourf(X0grid,X1grid,out.cpu().detach().numpy()[:,1].reshape((101,101)),alpha=0.5)
plt.scatter(X0[:,0],X0[:,1],c=y0)
plt.scatter(X0_test[:,0],X0_test[:,1],c=y0_test,marker='x')

This seems to have helped alot.  What happens if we adjust $\gamma$?  **Explore this value on your own, and determine what you think a sensible value is**.

## L2 Regularization for MNIST

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# In order to run this in class, we're going to reduce the dataset by a factor of 5
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=True)
X/=255.
y = y.astype(int)
X,X_test,y,y_test = train_test_split(X,y)
X = X
y = y

X = torch.from_numpy(X)
X_test = torch.from_numpy(X_test)
y = torch.from_numpy(y)
y_test = torch.from_numpy(y_test)

X = X.to(torch.float32)
X_test = X_test.to(torch.float32)
y = y.to(torch.long)
y_test = y_test.to(torch.long)

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

X = X.to(device)
X_test = X_test.to(device)
y = y.to(device)
y_test = y_test.to(device)

In [ ]:
from torch.utils.data import TensorDataset

training_data = TensorDataset(X,y)
test_data = TensorDataset(X_test,y_test)

batch_size = 256
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

batch_size = 256
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=batch_size, 
                                           shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        """
        This method is where you'll want to instantiate parameters.
        we do this by creating two linear transformation functions, l1 and l2, which 
        have encoded in it both the weight matrices W_1 and W_2, and the bias vectors
        """
        super(Net,self).__init__()
        self.l1 = nn.Linear(784,256) # Transform from input to hidden layer
        self.l2 = nn.Linear(256,256)
        self.l3 = nn.Linear(256,10)
        
        self.dropout_1 = nn.Dropout(p=0.3)
        self.dropout_2 = nn.Dropout(p=0.3)


    
    def forward(self,x):
        """
        This method runs the feedforward neural network.  It takes a tensor of size m x 784,
        applies a linear transformation, applies a sigmoidal activation, applies the second linear transform 
        and outputs the logits.
        """
        a1 = self.l1(x)
        z1 = torch.relu(a1)
        z1d = self.dropout_1(z1)
        
        a2 = self.l2(z1d)
        z2 = torch.relu(a2)
        z2d = self.dropout_2(z2)
       # 
        a3 = self.l3(z2d)

        return a3

In [ ]:
model = Net()
model.to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3,momentum=0.9)
gamma = 0

epochs = 200
# Loop over the data
for epoch in range(epochs):
    model.train()
    # Loop over each subset of data
    for d,t in train_loader:

        # Zero out the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Make a prediction based on the model
        outputs = model(d)
        
        # Compute the loss
        loss = criterion(outputs,t)
        for p in model.parameters():
            if p.dim()>1:
                loss += gamma/(2*d.shape[0])*(p**2).sum()
        

        # Use backpropagation to compute the derivative of the loss with respect to the parameters
        loss.backward()
        
        # Use the derivative information to update the parameters
        optimizer.step()
        
    model.eval()
    # After each epoch, compute the test set accuracy
    total=0.
    correct=0.
    # Loop over all the test examples and accumulate the number of correct results in each batch
    for d,t in test_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total += float(t.size(0))
        correct += float((predicted==t).sum())
    total_train = 0
    correct_train = 0
    for d,t in train_loader:
        outputs = model(d)
        _, predicted = torch.max(outputs.data,1)
        total_train += float(t.size(0))
        correct_train += float((predicted==t).sum())
        
    # Print the epoch, the training loss, and the test set accuracy.
    print(epoch,loss.item(),100.*correct_train/total_train,100.*correct/total)